In [837]:
# https://www.kaggle.com/wordcards/stock-market-tweets-wordcloud

In [1]:
# %load _header_import.py
import numpy as np 
import pandas as pd 
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 20)

import os
for dirname, _, filenames in os.walk('df_tweets_wordcloud/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

import warnings
warnings.simplefilter('ignore')
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from pprint import pprint
from datetime import datetime
import collections
import re

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from wordcloud import WordCloud
from collections import Counter        

# for autoreload modules
%load_ext autoreload
%autoreload 2


[nltk_data] Downloading package stopwords to /home/sergey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/sergey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/sergey/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# %load _load_target_data.py
path_data = '/mnt/files/workdata/work/python-scripts/prediction_analyzer/predict_stock_quotes/data/'

file_old = path_data + '21K-predict.csv'
df_old = pd.read_csv(file_old,  dtype=str)

print(df_old.shape)
# (21129, 8)

file_new = path_data + 'data-2021-06-10/trainingset _1_.xlsx'
df_new = pd.read_excel(file_new, dtype=str)

print(df_new.shape)
# (29006, 6)

df_old_sub = df_old[['title', 'Unnamed: 2']].copy()
df_old_sub.columns = ['text', 'SENTIMENT']

df_new_sub = df_new[['title', 'znak']].copy()
df_new_sub.columns = ['text', 'SENTIMENT']

mask = df_new_sub['text'].isin(df_old_sub['text'])
df_unique =  df_new_sub[~ mask].copy()
print(df_unique.shape[0])

# df_new_sub - dataset c новыми данными

# df_old_sub - исходный dataset
# df_unique - dataset с отобранными новыми данными

(21129, 8)
(29006, 6)
8078


### Prepare data

In [3]:
# %load _prepare_sentiment_data.py
_positive = 'positive'
_negative = 'negative'
_neutral = 'neutral'

sentiment_list = [_neutral, _positive, _negative]
replaced_dic = {'0': _neutral, '1': _positive, '2': _negative}

#  ---df_old_sub
df_old_sub['sentiment'] = df_old_sub['SENTIMENT'].replace(replaced_dic)
print('df_old_sub.shape= ', df_old_sub.shape[0])

mask = df_old_sub.sentiment.isin(sentiment_list)
df_old_sub = df_old_sub[mask].copy()
print('df_old_sub.shape with correct sentiment= ', df_old_sub.shape[0])

#  ---df_new_sub
df_new_sub['sentiment'] = df_new_sub['SENTIMENT'].replace(replaced_dic)
print('df_new_sub.shape =', df_new_sub.shape[0])

mask = df_new_sub.sentiment.isin(sentiment_list)
df_new_sub = df_new_sub[mask].copy()
print('df_new_sub.shape =', df_new_sub.shape[0])

# ---create df_unique
mask = df_new_sub['text'].isin(df_old_sub['text'])
df_unique =  df_new_sub[~ mask].copy()

print('df_unique.shape with correct sentiment= ', df_unique.shape[0])

url_pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')

def url(phrase):
    return url_pattern.sub('', phrase)

def prepare_data(dt: pd.DataFrame):
    mask = dt.text.notnull()
    dt = dt[mask].copy()

    dt['text'] = dt['text'].apply(url)

    dt.drop_duplicates(subset=['text'], keep='first', inplace=True)
    print(Counter(dt['sentiment']))
    
    return dt

# -- remove duplicate
print('df_old_sub.shape = ', df_old_sub.shape[0])
df_old_sub = prepare_data(df_old_sub)
print('df_old_sub.shape without duplicate = ', df_old_sub.shape[0])

print('df_unique.shape= ', df_unique.shape[0])
df_unique = prepare_data(df_unique)
print('df_unique.shape without duplicate = ', df_unique.shape[0])

df_old_sub.shape=  21129
df_old_sub.shape with correct sentiment=  20910
df_new_sub.shape = 29006
df_new_sub.shape = 28827
df_unique.shape with correct sentiment=  7988
df_old_sub.shape =  20910
Counter({'positive': 9305, 'neutral': 8452, 'negative': 2937})
df_old_sub.shape without duplicate =  20694
df_unique.shape=  7988
Counter({'neutral': 3545, 'positive': 3421, 'negative': 976})
df_unique.shape without duplicate =  7942


In [4]:
df_unique.head()

,text,SENTIMENT,sentiment
69,Has anyone seen this ‚ÄúRegulatory Fee?‚Äù what percentage is the fee? Can only be seen of you scroll down on a past sale order.,0,neutral
100,Can‚Äôt sell my dogecoin? Have equity but ‚Äú0.00 available‚Äù to sell.,0,neutral
180,FS KKR Capital¬† declares $0.55 dividend,1,positive
514,Schr√∂dinger Q1 Earnings Preview,1,positive
597,Nuveen California AMT-Free Municipal Income Fund¬† declares $0.0545 dividend,1,positive


In [5]:
df = df_old_sub

### -----------------------------Teach by body ------------------------

In [23]:
# df_new.head(1)

,created_at,title,znak,body,source,sentiment
0,22:20.6,ChipMOS misses on revenue,2,NaN,seeking-alpha,NaN


In [24]:
# df_new_body = df_new[['body', 'znak']].copy()
# print(df_new_body.shape[0])

# df_new_body.columns = ['text', 'SENTIMENT']
# mask = df_new_body.text.isnull()

# df_new_body = df_new_body[~mask]
# print(df_new_body.shape[0])

29006
11294


In [25]:
# df_new_body['sentiment'] = df_new_body['SENTIMENT'].replace(replaced_dic)
# mask = df_new_body.sentiment.isin(sentiment_list)

# df_new_body = df_new_body[mask].copy()
# print(df_new_body.shape[0])

# # -- remove duplicate
# df_new_body = prepare_data(df_new_body)
# print(df_new_body.shape[0])

11248
Counter({'positive': 4600, 'neutral': 4264, 'negative': 1441})
10305


In [26]:
# arrange_text(df_new_body)

In [28]:
# df_new_body.drop(columns='text', inplace=True)
# df_new_body.rename(columns={'text2':'text'},inplace=True)
# df_new_body.reset_index(drop=True, inplace=True)

In [36]:
# replaced_finbert = {_positive:0, _negative:1,_neutral:2}
# df_new_body['label'] = df_new_body['sentiment'].replace(replaced_finbert)

In [54]:
# df_new_body.head()

In [38]:
# comppiling_model(df_new_body, SGDClassifier, vectorize_method = 'TfidfVectorizer')#, seed=34

Model_name:  SGDClassifier(class_weight={0: 2.2548018477996594, 1: 7.161389961389961,
                            2: 2.3988618727366786})
Vectorize_method:  TfidfVectorizer
              precision    recall  f1-score   support

           0    0.87257   0.82957   0.85053       487
           1    0.78571   0.75342   0.76923       146
           2    0.81542   0.87688   0.84504       398

    accuracy                        0.83705      1031
   macro avg    0.82457   0.81996   0.82160      1031
weighted avg    0.83821   0.83705   0.83689      1031



(TfidfVectorizer(),
 SGDClassifier(class_weight={0: 2.2548018477996594, 1: 7.161389961389961,
                             2: 2.3988618727366786}),
 array([0, 0, 2, ..., 0, 0, 1]))

In [14]:
#              precision    recall  f1-score   support

#            0    0.86510   0.82957   0.84696       487
#            1    0.77083   0.76027   0.76552       146
#            2    0.81667   0.86181   0.83863       398

#     accuracy                        0.83220      1031
#    macro avg    0.81753   0.81722   0.81704      1031
# weighted avg    0.83305   0.83220   0.83221      1031

### ----------------------------- END Teach by body ------------------------

### PreProccess Text -------

In [39]:
from _arrange import arrange_text
# arrange create new clear column text2!!!!!

In [16]:
# df_old_sub - исходный dataset
# df_unique - dataset с отобранными новыми данными

In [40]:
arrange_text(df_old_sub)

In [41]:
df_old_sub.drop(columns='text', inplace=True)
df_old_sub.rename(columns={'text2':'text'},inplace=True)
df_old_sub.reset_index(drop=True, inplace=True)

In [42]:
arrange_text(df_unique)

In [43]:
df_unique.drop(columns='text', inplace=True)
df_unique.rename(columns={'text2':'text'},inplace=True)
df_unique.reset_index(drop=True, inplace=True)

In [44]:
df = df_old_sub 

In [22]:
stop

NameError: name 'stop' is not defined

### Quick view of preprocessed tweets

In [ ]:
# %load _word_cloud.py


### TEACHING =======================================================================================

In [ ]:
# https://github.com/DrManishSharma/NLP/blob/master/SentiAnalysis.ipynb

# https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html
# https://github.com/scikit-learn/scikit-learn/tree/main/doc/tutorial/text_analytics

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report
from sklearn.metrics import matthews_corrcoef

In [33]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier

In [34]:
def comppiling_model(dt: pd.DataFrame, CLS, vectorize_method: str, seed=2, threshhold=None, ngrams=1, extra_teach=None):    
    
    dt_train, dt_test = train_test_split(dt, test_size=0.1, random_state=seed)
    
    if extra_teach is not None:
        dt_train = pd.concat([dt_train, extra_teach])
    
    if vectorize_method == 'CountVectorizer':
        token = RegexpTokenizer(r'[a-zA-Z0-9]+')
        cv = CountVectorizer(stop_words='english', ngram_range = (ngrams, ngrams), tokenizer = token.tokenize)        
        text_counts = cv.fit_transform(dt['text'])
        
        X_train =cv.transform(dt_train['text']) 
        Y_train = dt_train['label']
        
        X_test = cv.transform(dt_test['text']) 
        Y_test = dt_test['label']
        
        
    else:
        tfidf = TfidfVectorizer()
        text_counts = tfidf.fit_transform(dt_train['text'])
        
                   
        X_train = tfidf.transform(dt_train['text']) 
        Y_train = dt_train['label']
        
        X_test = tfidf.transform(dt_test['text']) 
        Y_test = dt_test['label']

        
    n_train = dt_train.shape[0]
    class_weight = { k:n_train/v for k,v in Counter(dt_train['label']).items() }
    
    cls_instance = CLS(class_weight=class_weight)
    
    if  CLS == GaussianNB:
        cls_instance.fit(X_train.todense(), Y_train) 
    else:    
        cls_instance.fit(X_train, Y_train)
    
    if not threshhold:
        threshhold = 100 - Counter(Y_train)[1]/len(Y_train)*100
    
    threshold = 0
    if hasattr(cls_instance, 'predict_proba') and CLS != GaussianNB:
        predict_proba = cls_instance.predict_proba(X_test)[:,1]
        predicted_label = np.where(predict_proba > np.percentile(predict_proba, threshhold) , 1, 0)
        print(np.percentile(predict_proba, threshhold))
        
    elif hasattr(cls_instance, 'predict_proba') and CLS == GaussianNB:
        predict_proba = cls_instance.predict_proba(X_test.todense())[:,1]
        predicted_label = np.where(predict_proba > np.percentile(predict_proba, threshhold) , 1, 0)
        print(np.percentile(predict_proba, threshhold))
    else:
        predicted_label = cls_instance.predict(X_test)  
        
    print('Model_name: ', cls_instance)
    print('Vectorize_method: ', vectorize_method)
    
    if vectorize_method == 'CountVectorizer':
         print('Ngrams: ', ngrams)
    else: ngrams = 0
    
    print(classification_report(Y_test, predicted_label, digits=5))

    information = '%s %s %s' % (cls_instance, vectorize_method, ngrams)

    return tfidf, cls_instance, predicted_label

In [ ]:
# df_old_sub - исходный dataset
# df_unique - dataset с отобранными новыми данными

In [45]:
replaced_finbert = {_positive:0, _negative:1, _neutral:2}

df_old_sub['label'] = df_old_sub['sentiment'].replace(replaced_finbert)
df_unique['label'] = df_unique['sentiment'].replace(replaced_finbert)

In [46]:
dt_train, dt_test = train_test_split(df_old_sub, test_size=0.1, random_state=2)

In [47]:
print(Counter(df_old_sub['label']))
print(Counter(df_old_sub['sentiment']))

Counter({0: 9305, 2: 8452, 1: 2937})
Counter({'positive': 9305, 'neutral': 8452, 'negative': 2937})


In [48]:
n_train = dt_train.shape[0]
class_weight_train = { k:n_train/v for k,v in Counter(dt_train['label']).items() }
class_weight_train

{2: 2.4354648881914476, 1: 7.089455652835935, 0: 2.2304191616766467}

In [49]:
n_unique = df_unique.shape[0]
class_weight_df_unique = { k:n_train/v for k,v in Counter(df_unique['label']).items() }
class_weight_df_unique

{2: 5.253596614950634, 0: 5.444022215726396, 1: 19.081967213114755}

In [53]:
tf_idf, cls_instance, predicted_label = comppiling_model(df, SGDClassifier, vectorize_method = 'TfidfVectorizer', extra_teach=df_unique)


Model_name:  SGDClassifier(class_weight={0: 2.2569025571319346, 1: 7.373300027754649,
                            2: 2.3736597569692637})
Vectorize_method:  TfidfVectorizer
              precision    recall  f1-score   support

           0    0.85865   0.85236   0.85549       955
           1    0.81250   0.75484   0.78261       310
           2    0.81775   0.84720   0.83221       805

    accuracy                        0.83575      2070
   macro avg    0.82963   0.81813   0.82344      2070
weighted avg    0.83583   0.83575   0.83552      2070



In [ ]:
# Model_name:  SGDClassifier(class_weight={0: 2.2569025571319346, 1: 7.373300027754649,
#                             2: 2.3736597569692637})
# Vectorize_method:  TfidfVectorizer
#               precision    recall  f1-score   support

#            0    0.85654   0.85026   0.85339       955
#            1    0.81185   0.75161   0.78057       310
#            2    0.81677   0.84720   0.83171       805

#     accuracy                        0.83430      2070
#    macro avg    0.82838   0.81636   0.82189      2070
# weighted avg    0.83438   0.83430   0.83405      2070

In [ ]:
comppiling_model(df, SGDClassifier, vectorize_method = 'TfidfVectorizer')

In [ ]:
# Model_name:  SGDClassifier(class_weight={0: 2.2569025571319346, 1: 7.373300027754649,
#                             2: 2.3736597569692637})
# Vectorize_method:  TfidfVectorizer
#               precision    recall  f1-score   support

#            0    0.85654   0.85026   0.85339       955
#            1    0.81185   0.75161   0.78057       310
#            2    0.81677   0.84720   0.83171       805

#     accuracy                        0.83430      2070
#    macro avg    0.82838   0.81636   0.82189      2070
# weighted avg    0.83438   0.83430   0.83405      2070

### Save models

In [ ]:
from datetime import date
today = date.today()
today = today.strftime("%Y-%m-%d")
today

In [ ]:
model_tf_idf_path = 'models/sgd_models/tf_idf_{}'.format(today)
model_sgd_classifier_path = 'models/sgd_models/sgd_classifier_multiclass_{}'.format(today)

print(model_tf_idf_path)
print(model_sgd_classifier_path)
# tf_idf, cls_instance, predicted_label = comppiling_model(df, SGDClassifier, vectorize_method = 'TfidfVectorizer')

In [ ]:
import joblib
# joblib.dump(tf_idf, model_tf_idf_path)
# joblib.dump(cls_instance, model_sgd_classifier_path )

### Load models

In [ ]:
# df_old_sub - исходный dataset
# df_unique - dataset с отобранными новыми данными

In [ ]:
import joblib

In [ ]:

tf_idf = joblib.load(model_tf_idf_path)
cls_instance = joblib.load(model_sgd_classifier_path)

##### Проверка результатов сохраненной модели на тестовом сете

In [ ]:
X_test = tf_idf.transform(dt_test['text']) 
Y_test = dt_test['label']

predicted_label = cls_instance.predict(X_test)
print(classification_report(Y_test, predicted_label, digits=5))

### retrain the model( дообучать слабую модель смысла нет)

In [ ]:
cls_instance.class_weight

In [ ]:
tf_idf.fit_transform(df_unique['text'])

In [ ]:
X_train = tf_idf.transform(df_unique['text']) 
Y_train = df_unique['label']

X_test = tf_idf.transform(dt_test['text']) 
Y_test = dt_test['label']

n_train = df_unique.shape[0]
class_weight = { k:n_train/v for k,v in Counter(df_unique['label']).items() }
print('class weight for df_unique =', class_weight)

In [ ]:
cls_instance.fit(X_train, Y_train)

#### ===================================== Compare results =====================================

In [ ]:
predicted_label = cls_instance.predict(X_test)

In [ ]:
print(classification_report(Y_test, predicted_label, digits=5))

In [ ]:
# class_weight_train
# {1: 7.145015105740181, 0: 2.2263313609467454, 2: 2.43384298735666}

# Model_name:  SGDClassifier(class_weight={0: 2.2263313609467454, 1: 7.145015105740181,
#                             2: 2.43384298735666})
# Vectorize_method:  TfidfVectorizer
#               precision    recall  f1-score   support

#            0    0.85125   0.83895   0.84506      1869
#            1    0.80909   0.71774   0.76068       620
#            2    0.80252   0.84970   0.82543      1650

#     accuracy                        0.82508      4139
#    macro avg    0.82095   0.80213   0.81039      4139
# weighted avg    0.82551   0.82508   0.82459      4139

### Stop

In [ ]:
# class_names = [MultinomialNB, ComplementNB, GaussianNB, BernoulliNB, LinearSVC, SGDClassifier]

In [ ]:
# def get_predicted_label(label_positive, label_negative):
#     if label_positive == 1 and label_negative == 0:
#         return 1
    
#     if label_positive == 0 and label_negative == 1:
#         return 2
#     return 0

# replaced_dic = {'0': _neutral, '1': _positive, '2': _negative, '3': 'unknown'}
# replaced_dic_finbert= {_positive: 0, _neutral: 1, _negative: 2}
# dt_test['label_num'] = dt_test['sentiment'].replace(replaced_dic_num)



# all_predicted_label_num = []
# for i in range(dt_test.shape[0]):
#     predicted_label_num = get_predicted_label(dt_test['predicted_label_positive'].values[i], dt_test['predicted_label_negative'].values[i])
#     all_predicted_label_num.append(predicted_label_num)
    
    
# dt_test['predicted_label_num'] = all_predicted_label_num

# print(Counter(dt_test['sentiment']))
# print(Counter(dt_test['label_num'])) 

# print(Counter(dt_test.predicted_label_positive))
# print(Counter(dt_test['predicted_label_num']))

# print(classification_report(dt_test['label_num'].values, dt_test['predicted_label_num'].values, digits=5))

In [ ]:
class_names = [MultinomialNB, ComplementNB, GaussianNB, BernoulliNB, LinearSVC, SGDClassifier]

vectorize_methods = ['CountVectorizer', 'TfidfVectorizer']
ngrames = [1, 2, 3]

all_info = []
all_coefs = []

for class_name in class_names:
    result = comppiling_model(df, class_name, vectorize_method = 'TfidfVectorizer')
    all_info.append(result[0])
    all_coefs.append(result[1])
    for ngram in ngrames:
        result = comppiling_model(df, class_name, vectorize_method = 'CountVectorizer', ngrams=ngram)
        all_info.append(result[0])
        all_coefs.append(result[1])  

In [ ]:
all_results = pd.DataFrame(all_matthews_coefs, all_info, columns=['matthew_coef']).sort_values(by='matthew_coef', ascending=False)

In [ ]:
all_results.head()

### Results without some short and stop words

#### positive - MultinomialNB() TfidfVectorizer 0, matthew_coef = 0.531
#### negative - SGDClassifier() CountVectorizer 3, matthew_coef = 0.399
#### neutral -  MultinomialNB() TfidfVectorizer 0, matthew_coef = 0.471

In [ ]:
# data = {'pred': predict_proba, 'label':Y_test }
# tmp = pd.DataFrame(data)
# tmp.describe()

In [ ]:
print(stop)

### Sentiment model (optional)

In [ ]:
import requests
import json

In [ ]:
url = 'http://127.0.0.1:8000/api/v0/predict'
payload = '{"text": "good news"}' 
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

response = requests.post(url, data=payload, headers=headers)
json_data = json.loads(response.text)

In [ ]:
df_sub.head()

In [ ]:
all_sentiment_title = []
all_sentiment_score_title = []
for title in df_sub['text'].values:
#     print(title)

    payload = json.dumps({"text": title}) 

    response = requests.post(url, data=payload, headers=headers)
    json_data = json.loads(response.text)
#     print(json_data)    
    all_sentiment_title.append(json_data['sentiment'])
    all_sentiment_score_title.append(json_data['sentiment_score'])

df_sub['pred_sentiment_title']  = all_sentiment_title
df_sub['pred_sentiment_score_title']  = all_sentiment_score_title    

In [ ]:
all_sentiment_title = []
all_sentiment_score_title = []
for title in df_sub['text_back'].values:
#     print(title)

    payload = json.dumps({"text": title}) 

    response = requests.post(url, data=payload, headers=headers)
    json_data = json.loads(response.text)
#     print(json_data)    
    all_sentiment_title.append(json_data['sentiment'])
    all_sentiment_score_title.append(json_data['sentiment_score'])
    
df_sub['back_pred_sentiment_title']  = all_sentiment_title
df_sub['back_pred_sentiment_score_title']  = all_sentiment_score_title
    

In [ ]:
df_sub['back_pred_sentiment_title'].value_counts()

In [ ]:
df_sub['sentiment'].value_counts()/df_sub.shape[0]

In [ ]:
df_sub['pred_sentiment_title'].value_counts()/df_sub.shape[0]

In [ ]:
train, test = train_test_split(df_sub, test_size=0.2,random_state=2)

In [ ]:
test.sentiment.value_counts()

In [ ]:
label = np.where(test.sentiment == _neutral, 1, 0)
predicted_label = np.where(test.back_pred_sentiment_title == _neutral, 1, 0)
print(classification_report(label, predicted_label, digits=5))

In [ ]:
d = {"text":['djlfjlsajdfla']} 
dt = pd.DataFrame(d)    

In [ ]:
dt

In [ ]:
>>> import numpy as np
>>> from sklearn.linear_model import SGDClassifier
>>> from sklearn.preprocessing import StandardScaler
>>> from sklearn.pipeline import make_pipeline
>>> X = np.array([[-1, -1], [-2, -1], [1, 1], [2, 1],[-1, 51]])
>>> Y = np.array([1, 0, 2, 0,1])
>>> # Always scale the input. The most convenient way is to use a pipeline.
>>> clf = make_pipeline(StandardScaler(),
...                     SGDClassifier(max_iter=1000, tol=1e-3))
>>> clf.fit(X, Y)

>>> print(clf.predict([[-2, -1]]))